# Table of Content
01. Import Libraries
02. Import Data
03. Prepare A List of Unique Addresses
04. Retrieve 1000 Data from OneMap API<br>
04.1 Using requests.get()<br>
04.2 Using requests with session<br>
04.3 Using aiohttp
05. Retrieve 9804 Data from OneMap API<br>
05.1 Using aiohttp - get error at the end<br>
05.2 Solution - Using aiohttp with Semaphore (rate limiter)

# 01. Import Libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import requests  # Import requests to connect to an API and perform actions on it
import asyncio     # the library needed for asynchronous operation
import nest_asyncio     # the library that allows nested use of the asyncio event loop, making asyncio compatible with Jupyter notebooks
import aiohttp     # the library that allows asychronous API calls
import time        # time library is used to handle time-related tasks

In [2]:
# Allow nested asyncio calls in Jupyter notebooks
nest_asyncio.apply()

In [3]:
# Adjust setting to allow seeing all the rows in the output of this notebook
pd.options.display.max_rows = None

In [4]:
# Adjust setting to allow seeing all the columns in the output of this notebook
pd.options.display.max_columns = None

# 02. Import Data

In [5]:
# Define the main project folder path
path = r'C:\Users\saich\Desktop\CareerFoundry\Data Immersion\Achievement 6 Advanced Analytics & Dashboard Design\11-2023 HDB Flat Resale Analysis'

In [6]:
flat_resale = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'flat_resale_all (checked).csv'), index_col = 0)

In [7]:
flat_resale.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,row_id
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,Improved,1977,9000.0,86,0
1,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,Improved,1977,6000.0,86,1
2,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,Improved,1977,8000.0,86,2
3,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,Improved,1977,6000.0,86,3
4,1990-01-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,New Generation,1976,47200.0,85,4


In [8]:
flat_resale.shape

(915371, 12)

# 03. Prepare a List of Unique Addresses

### 03.1 Add Address Column

To get coordinates information from OneMap API, a complete address is needed for accurate search.

Address = block + street name

In [9]:
flat_resale['address'] = flat_resale['block'] + ' ' + flat_resale['street_name']

In [10]:
flat_resale.head(10)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,row_id,address
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,Improved,1977,9000.0,86,0,309 ANG MO KIO AVE 1
1,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,Improved,1977,6000.0,86,1,309 ANG MO KIO AVE 1
2,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,Improved,1977,8000.0,86,2,309 ANG MO KIO AVE 1
3,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,Improved,1977,6000.0,86,3,309 ANG MO KIO AVE 1
4,1990-01-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,New Generation,1976,47200.0,85,4,216 ANG MO KIO AVE 1
5,1990-01-01,ANG MO KIO,3 ROOM,211,ANG MO KIO AVE 3,01 TO 03,67.0,New Generation,1977,46000.0,86,5,211 ANG MO KIO AVE 3
6,1990-01-01,ANG MO KIO,3 ROOM,202,ANG MO KIO AVE 3,07 TO 09,67.0,New Generation,1977,42000.0,86,6,202 ANG MO KIO AVE 3
7,1990-01-01,ANG MO KIO,3 ROOM,235,ANG MO KIO AVE 3,10 TO 12,67.0,New Generation,1977,38000.0,86,7,235 ANG MO KIO AVE 3
8,1990-01-01,ANG MO KIO,3 ROOM,235,ANG MO KIO AVE 3,04 TO 06,67.0,New Generation,1977,40000.0,86,8,235 ANG MO KIO AVE 3
9,1990-01-01,ANG MO KIO,3 ROOM,232,ANG MO KIO AVE 3,01 TO 03,67.0,New Generation,1977,47000.0,86,9,232 ANG MO KIO AVE 3


In [11]:
flat_resale.shape

(915371, 13)

### 03.2 Extract A List of Unique Addresses

In [12]:
addresses = flat_resale['address'].unique()

In [13]:
addresses

array(['309 ANG MO KIO AVE 1', '216 ANG MO KIO AVE 1',
       '211 ANG MO KIO AVE 3', ..., '673C YISHUN AVE 4',
       '468A YISHUN ST 43', '674B YISHUN AVE 4'], dtype=object)

In [14]:
len(addresses)

9804

# 04. Get Coordinates of 1000 Flats from OneMap API

In [15]:
# The URL that connects with OneMap API
url = 'https://www.onemap.gov.sg/api/common/elastic/search?searchVal={}&returnGeom=Y&getAddrDetails=Y&pageNum=1'

In [16]:
# Testing

# Connect with OneMap API with get() function, get() function is used to get the information from the API
response = requests.get(url.format(addresses[0]))
# Parse the JSON response using .json() method 
data = response.json()
data

{'found': 10000,
 'totalNumPages': 1000,
 'pageNum': 1,
 'results': [{'SEARCHVAL': 'ANG MO KIO 22',
   'BLK_NO': '216',
   'ROAD_NAME': 'ANG MO KIO AVENUE 1',
   'BUILDING': 'ANG MO KIO 22',
   'ADDRESS': '216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPORE 560216',
   'POSTAL': '560216',
   'X': '28911.0522399368',
   'Y': '38692.6167905722',
   'LATITUDE': '1.36619678831054',
   'LONGITUDE': '103.841505011903'},
  {'SEARCHVAL': 'ANG MO KIO 22',
   'BLK_NO': '207',
   'ROAD_NAME': 'ANG MO KIO AVENUE 1',
   'BUILDING': 'ANG MO KIO 22',
   'ADDRESS': '207 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPORE 560207',
   'POSTAL': '560207',
   'X': '29060.4855782297',
   'Y': '38651.0529769398',
   'LATITUDE': '1.36582089504381',
   'LONGITUDE': '103.842847772078'},
  {'SEARCHVAL': 'ANG MO KIO 22',
   'BLK_NO': '215',
   'ROAD_NAME': 'ANG MO KIO AVENUE 1',
   'BUILDING': 'ANG MO KIO 22',
   'ADDRESS': '215 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPORE 560215',
   'POSTAL': '560215',
   'X': '28924.30329

### 04.1 Using requests.get()

In [17]:
# Create a function to get coordinates using requests
def get_coordinates_requests(address, count): 
    # Connect with OneMap API with get() function, get() function is used to get the information from the API
    response = requests.get(url.format(address))
    # Parse the JSON response using .json() method 
    data = response.json()
    # Print the result
    print('{}) {}'.format(count, data['results'][0]['ADDRESS']))

In [18]:
count = 1

# 'start' here indicates the starting time of the retrieval process
start = time.time()

for address in addresses[:1000]: 
    get_coordinates_requests(address, count)
    count +=1
    
# 'end' here indicates the ending time of the retrieval process
end = time.time()

print('Total time used: {} seconds'.format(end - start))

1) 216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPORE 560216
2) 216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPORE 560216
3) 211 ANG MO KIO AVENUE 3 SINGAPORE 560211
4) 202 ANG MO KIO AVENUE 3 ANG MO KIO VIEW SINGAPORE 560202
5) 235 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW SINGAPORE 560235
6) 232 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW SINGAPORE 560232
7) 308C ANG MO KIO AVENUE 1 TECK GHEE VISTA SINGAPORE 563308
8) 220 ANG MO KIO AVENUE 1 ANG MO KIO GROVE SINGAPORE 560220
9) 219 ANG MO KIO AVENUE 1 ANG MO KIO GROVE SINGAPORE 560219
10) 329 ANG MO KIO AVENUE 3 ANG MO KIO 31 SINGAPORE 560329
11) 320 ANG MO KIO AVENUE 1 STAR LEARNERS @ ANG MO KIO PTE. LTD. SINGAPORE 560320
12) 641 ANG MO KIO AVENUE 4 ANG MO KIO 61 SINGAPORE 560641
13) 223 ANG MO KIO AVENUE 1 SINGAPORE 560223
14) 230 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW SINGAPORE 560230
15) 329 ANG MO KIO AVENUE 3 ANG MO KIO 31 SINGAPORE 560329
16) 313 ANG MO KIO AVENUE 3 TECK GHEE EVERGREEN SINGAPORE 560313
17) 117 ANG MO KIO AVENUE 4 KEBU

138) 107 BEDOK NORTH ROAD FENGSHAN ESTATE SINGAPORE 460107
139) 110 BEDOK NORTH ROAD FENGSHAN ESTATE SINGAPORE 460110
140) 69 BEDOK SOUTH AVENUE 3 SIGLAP LUCKY VALLEY SINGAPORE 460069
141) 33 BEDOK SOUTH AVENUE 2 SINGAPORE 460033
142) 46 BEDOK SOUTH AVENUE 3 SINGAPORE 460046
143) 117 BEDOK NORTH ROAD SINGAPORE 460117
144) 116 BEDOK NORTH ROAD SINGAPORE 460116
145) 115 BEDOK NORTH ROAD SINGAPORE 460115
146) 125 BEDOK NORTH ROAD SINGAPORE 460125
147) 124 BEDOK NORTH ROAD SINGAPORE 460124
148) 128 BEDOK NORTH STREET 2 SINGAPORE 460128
149) 129 BEDOK NORTH STREET 2 SINGAPORE 460129
150) 138 BEDOK NORTH STREET 2 SINGAPORE 460138
151) 87 BEDOK NORTH STREET 4 SINGAPORE 460087
152) 100 BEDOK NORTH AVENUE 4 BEDOK POND 1 SINGAPORE 460100
153) 412 BEDOK NORTH AVENUE 2 SINGAPORE 460412
154) 402 BEDOK NORTH AVENUE 3 RAINBOW VILLE SINGAPORE 460402
155) 401 BEDOK NORTH AVENUE 3 RAINBOW VILLE SINGAPORE 460401
156) 425 BEDOK NORTH ROAD GARDEN HILL SINGAPORE 460425
157) 416 BEDOK NORTH AVENUE 2 GARDEN H

290) 216 BUKIT BATOK STREET 21 SINGAPORE 650216
291) 115 BUKIT BATOK WEST AVENUE 6 SINGAPORE 650115
292) 117 BUKIT BATOK WEST AVENUE 6 SINGAPORE 650117
293) 111 BUKIT BATOK WEST AVENUE 6 SINGAPORE 650111
294) 103 BUKIT BATOK CENTRAL SINGAPORE 650103
295) 223 BUKIT BATOK EAST AVENUE 3 SINGAPORE 650223
296) 221 BUKIT BATOK EAST AVENUE 3 SINGAPORE 650221
297) 214 BUKIT BATOK STREET 21 SINGAPORE 650214
298) 120 BUKIT BATOK CENTRAL SINGAPORE 650120
299) 121 BUKIT BATOK CENTRAL SINGAPORE 650121
300) 142 BUKIT BATOK STREET 11 SINGAPORE 650142
301) 140 BUKIT BATOK STREET 11 SINGAPORE 650140
302) 141 BUKIT BATOK STREET 11 SINGAPORE 650141
303) 12A TAMAN HO SWEE BUKIT HO SWEE VIEW SINGAPORE 164012
304) 7 TELOK BLANGAH CRESCENT SINGAPORE 090007
305) 40 BEO CRESCENT THE BEO CRESCENT SINGAPORE 160040
306) 44 TELOK BLANGAH DRIVE BLANGAH COURT SINGAPORE 100044
307) 46 TELOK BLANGAH DRIVE BLANGAH COURT SINGAPORE 100046
308) 105C DEPOT ROAD SINGAPORE 103105
309) 110 DEPOT ROAD DEPOT HEIGHTS SINGAPORE 1

443) 602 CLEMENTI WEST STREET 1 SAINT LUKE'S ELDERCARE CLEMENTI CENTRE SINGAPORE 120602
444) 607 CLEMENTI WEST STREET 1 SINGAPORE 120607
445) 608 CLEMENTI WEST STREET 1 SINGAPORE 120608
446) 702 WEST COAST ROAD SINGAPORE 120702
447) 703 WEST COAST ROAD SINGAPORE 120703
448) 704 WEST COAST ROAD SINGAPORE 120704
449) 711 CLEMENTI WEST STREET 2 SINGAPORE 120711
450) 705 CLEMENTI WEST STREET 2 SINGAPORE 120705
451) 713 CLEMENTI WEST STREET 2 SINGAPORE 120713
452) 731 CLEMENTI WEST STREET 2 SINGAPORE 120731
453) 728 CLEMENTI WEST STREET 2 SINGAPORE 120728
454) 706 CLEMENTI WEST STREET 2 SINGAPORE 120706
455) 710 CLEMENTI WEST STREET 2 SINGAPORE 120710
456) 610 CLEMENTI WEST STREET 1 SINGAPORE 120610
457) 113 CLEMENTI STREET 13 SUNSET WAY RESIDENCE SINGAPORE 120113
458) 412 COMMONWEALTH AVENUE WEST SINGAPORE 120412
459) 411 COMMONWEALTH AVENUE WEST SINGAPORE 120411
460) 315 CLEMENTI AVENUE 4 CLEMENTI MEADOWS SINGAPORE 120315
461) 311 CLEMENTI AVENUE 4 TRIVELIS SINGAPORE 120311
462) 301 CLEME

613) 408 PANDAN GARDENS SINGAPORE 600408
614) 218 JURONG EAST STREET 21 SINGAPORE 600218
615) 322 JURONG EAST STREET 31 SINGAPORE 600322
616) 227 JURONG EAST STREET 21 SINGAPORE 600227
617) 109 JURONG EAST STREET 13 JURONG EAST VILLE SINGAPORE 600109
618) 103 JURONG EAST STREET 13 JURONG EAST VILLE SINGAPORE 600103
619) 240 JURONG EAST STREET 21 SINGAPORE 600240
620) 258 JURONG EAST STREET 24 SINGAPORE 600258
621) 209 JURONG EAST STREET 21 SINGAPORE 600209
622) 324 JURONG EAST STREET 31 SINGAPORE 600324
623) 331 JURONG EAST AVENUE 1 SINGAPORE 600331
624) 405 PANDAN GARDENS SINGAPORE 600405
625) 403 PANDAN GARDENS SINGAPORE 600403
626) 404 PANDAN GARDENS SINGAPORE 600404
627) 406 PANDAN GARDENS SINGAPORE 600406
628) 337 JURONG EAST AVENUE 1 SINGAPORE 600337
629) 335 JURONG EAST AVENUE 1 SINGAPORE 600335
630) 327 JURONG EAST STREET 31 SAINT LUKE'S ELDERCARE JURONG EAST CENTRE SINGAPORE 600327
631) 323 JURONG EAST STREET 31 SINGAPORE 600323
632) 164 YUNG KUANG ROAD YUNG KUANG COURT SINGAP

765) 30 MARINE CRESCENT MARINE CRESCENT VILLE SINGAPORE 440030
766) 7 MARINE TERRACE MARINE TERRACE WALK SINGAPORE 440007
767) 62 MARINE DRIVE MARINE DRIVE GARDENS SINGAPORE 440062
768) 55 MARINE TERRACE MARINE TERRACE HAVEN SINGAPORE 440055
769) 43 MARINE CRESCENT MARINE CRESCENT GARDENS SINGAPORE 440043
770) 73 MARINE DRIVE MARINE DRIVE VIEW SINGAPORE 440073
771) 72 MARINE DRIVE MARINE DRIVE VIEW SINGAPORE 440072
772) 71 MARINE DRIVE MARINE DRIVE GARDENS SINGAPORE 440071
773) 78 MARINE DRIVE MARINE DRIVE VIEW SINGAPORE 440078
774) 3 MARINE TERRACE MARINE TERRACE WALK SINGAPORE 440003
775) 12 MARINE TERRACE MARINE TERRACE BREEZE SINGAPORE 440012
776) 13 MARINE TERRACE MARINE TERRACE BREEZE SINGAPORE 440013
777) 14 MARINE TERRACE MARINE TERRACE BREEZE SINGAPORE 440014
778) 82 COMMONWEALTH CLOSE COMMONWEALTH HEIGHTS SINGAPORE 140082
779) 83 COMMONWEALTH CLOSE COMMONWEALTH HEIGHTS SINGAPORE 140083
780) 63 COMMONWEALTH DRIVE COMMONWEALTH GREEN SINGAPORE 140063
781) 66 COMMONWEALTH DRIVE C

918) 810 TAMPINES AVENUE 4 TAMPINES POLYVIEW SINGAPORE 520810
919) 266 TAMPINES STREET 21 SINGAPORE 520266
920) 265 TAMPINES STREET 21 SINGAPORE 520265
921) 286 TAMPINES STREET 22 SINGAPORE 520286
922) 272 TAMPINES STREET 22 SINGAPORE 520272
923) 275 TAMPINES STREET 22 SINGAPORE 520275
924) 141 TAMPINES STREET 12 SINGAPORE 521141
925) 163 TAMPINES STREET 12 SINGAPORE 521163
926) 166 TAMPINES STREET 12 SINGAPORE 521166
927) 161 TAMPINES STREET 12 SINGAPORE 521161
928) 295 TAMPINES STREET 22 SINGAPORE 520295
929) 261 TAMPINES STREET 21 SINGAPORE 520261
930) 913 TAMPINES STREET 91 TAMPINES PALMSVILLE SINGAPORE 520913
931) 910 TAMPINES STREET 91 TAMPINES PALMSVILLE SINGAPORE 520910
932) 253 TAMPINES STREET 21 SINGAPORE 521253
933) 277 TAMPINES STREET 22 SINGAPORE 520277
934) 928 TAMPINES STREET 91 TAMPINES PALMSPRING SINGAPORE 520928
935) 832 TAMPINES STREET 82 TAMPINES PALMWALK SINGAPORE 520832
936) 838 TAMPINES STREET 82 TAMPINES PALMWALK SINGAPORE 520838
937) 839 TAMPINES STREET 83 TAMP

<b>Total time used to retrieve 1000 addresses: 568 seconds</b>

### 04.2 Using requests session

In [19]:
# Create a function to get coordinates using requests session
def get_coordinates_requests_session(session, address, count): 
    # Connect with OneMap API with get() function, get() function is used to get the information from the API
    response = session.get(url.format(address))
    # Pull the data from OneMap API and store it into 'data'
    data = response.json()
    # Print the result
    print('{}) {}'.format(count, data['results'][0]['ADDRESS']))

In [20]:
count = 1

# 'start' here indicates the starting time of the retrieval process
start = time.time()

# A Session is used so that it could reuse the underlying TCP connection for all requests
# Sessions can also be used as context managers (like how we write here)
# This will make sure the session is closed as soon as the with block is exited
with requests.Session() as session: 
    
    for address in addresses[:1000]: 
        get_coordinates_requests_session(session, address, count)
        count +=1

# 'end' here indicates the ending time of the retrieval process
end = time.time()

print('Total time used: {} seconds'.format(end - start))

1) 216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPORE 560216
2) 216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPORE 560216
3) 211 ANG MO KIO AVENUE 3 SINGAPORE 560211
4) 202 ANG MO KIO AVENUE 3 ANG MO KIO VIEW SINGAPORE 560202
5) 235 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW SINGAPORE 560235
6) 232 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW SINGAPORE 560232
7) 308C ANG MO KIO AVENUE 1 TECK GHEE VISTA SINGAPORE 563308
8) 220 ANG MO KIO AVENUE 1 ANG MO KIO GROVE SINGAPORE 560220
9) 219 ANG MO KIO AVENUE 1 ANG MO KIO GROVE SINGAPORE 560219
10) 329 ANG MO KIO AVENUE 3 ANG MO KIO 31 SINGAPORE 560329
11) 320 ANG MO KIO AVENUE 1 STAR LEARNERS @ ANG MO KIO PTE. LTD. SINGAPORE 560320
12) 641 ANG MO KIO AVENUE 4 ANG MO KIO 61 SINGAPORE 560641
13) 223 ANG MO KIO AVENUE 1 SINGAPORE 560223
14) 230 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW SINGAPORE 560230
15) 329 ANG MO KIO AVENUE 3 ANG MO KIO 31 SINGAPORE 560329
16) 313 ANG MO KIO AVENUE 3 TECK GHEE EVERGREEN SINGAPORE 560313
17) 117 ANG MO KIO AVENUE 4 KEBU

141) 33 BEDOK SOUTH AVENUE 2 SINGAPORE 460033
142) 46 BEDOK SOUTH AVENUE 3 SINGAPORE 460046
143) 117 BEDOK NORTH ROAD SINGAPORE 460117
144) 116 BEDOK NORTH ROAD SINGAPORE 460116
145) 115 BEDOK NORTH ROAD SINGAPORE 460115
146) 125 BEDOK NORTH ROAD SINGAPORE 460125
147) 124 BEDOK NORTH ROAD SINGAPORE 460124
148) 128 BEDOK NORTH STREET 2 SINGAPORE 460128
149) 129 BEDOK NORTH STREET 2 SINGAPORE 460129
150) 138 BEDOK NORTH STREET 2 SINGAPORE 460138
151) 87 BEDOK NORTH STREET 4 SINGAPORE 460087
152) 100 BEDOK NORTH AVENUE 4 BEDOK POND 1 SINGAPORE 460100
153) 412 BEDOK NORTH AVENUE 2 SINGAPORE 460412
154) 402 BEDOK NORTH AVENUE 3 RAINBOW VILLE SINGAPORE 460402
155) 401 BEDOK NORTH AVENUE 3 RAINBOW VILLE SINGAPORE 460401
156) 425 BEDOK NORTH ROAD GARDEN HILL SINGAPORE 460425
157) 416 BEDOK NORTH AVENUE 2 GARDEN HILL SINGAPORE 460416
158) 418 BEDOK NORTH AVENUE 2 GARDEN HILL SINGAPORE 460418
159) 424 BEDOK NORTH AVENUE 1 GARDEN HILL SINGAPORE 460424
160) 419 BEDOK NORTH STREET 1 GARDEN HILL SIN

295) 223 BUKIT BATOK EAST AVENUE 3 SINGAPORE 650223
296) 221 BUKIT BATOK EAST AVENUE 3 SINGAPORE 650221
297) 214 BUKIT BATOK STREET 21 SINGAPORE 650214
298) 120 BUKIT BATOK CENTRAL SINGAPORE 650120
299) 121 BUKIT BATOK CENTRAL SINGAPORE 650121
300) 142 BUKIT BATOK STREET 11 SINGAPORE 650142
301) 140 BUKIT BATOK STREET 11 SINGAPORE 650140
302) 141 BUKIT BATOK STREET 11 SINGAPORE 650141
303) 12A TAMAN HO SWEE BUKIT HO SWEE VIEW SINGAPORE 164012
304) 7 TELOK BLANGAH CRESCENT SINGAPORE 090007
305) 40 BEO CRESCENT THE BEO CRESCENT SINGAPORE 160040
306) 44 TELOK BLANGAH DRIVE BLANGAH COURT SINGAPORE 100044
307) 46 TELOK BLANGAH DRIVE BLANGAH COURT SINGAPORE 100046
308) 105C DEPOT ROAD SINGAPORE 103105
309) 110 DEPOT ROAD DEPOT HEIGHTS SINGAPORE 100110
310) 42 TELOK BLANGAH RISE SINGAPORE 090042
311) 132 JALAN BUKIT MERAH SINGAPORE 160132
312) 133 JALAN BUKIT MERAH SINGAPORE 160133
313) 96 HENDERSON ROAD CITY VUE @HENDERSON SINGAPORE 150096
314) 77 INDUS ROAD INDUS GARDEN SINGAPORE 160077
315

449) 711 CLEMENTI WEST STREET 2 SINGAPORE 120711
450) 705 CLEMENTI WEST STREET 2 SINGAPORE 120705
451) 713 CLEMENTI WEST STREET 2 SINGAPORE 120713
452) 731 CLEMENTI WEST STREET 2 SINGAPORE 120731
453) 728 CLEMENTI WEST STREET 2 SINGAPORE 120728
454) 706 CLEMENTI WEST STREET 2 SINGAPORE 120706
455) 710 CLEMENTI WEST STREET 2 SINGAPORE 120710
456) 610 CLEMENTI WEST STREET 1 SINGAPORE 120610
457) 113 CLEMENTI STREET 13 SUNSET WAY RESIDENCE SINGAPORE 120113
458) 412 COMMONWEALTH AVENUE WEST SINGAPORE 120412
459) 411 COMMONWEALTH AVENUE WEST SINGAPORE 120411
460) 315 CLEMENTI AVENUE 4 CLEMENTI MEADOWS SINGAPORE 120315
461) 311 CLEMENTI AVENUE 4 TRIVELIS SINGAPORE 120311
462) 301 CLEMENTI AVENUE 4 CLEMENTI HAVEN SINGAPORE 120301
463) 609 CLEMENTI WEST STREET 1 SINGAPORE 120609
464) 729 CLEMENTI WEST STREET 2 SINGAPORE 120729
465) 707 CLEMENTI WEST STREET 2 SINGAPORE 120707
466) 371 CLEMENTI AVENUE 4 SINGAPORE 120371
467) 205 CLEMENTI AVENUE 6 SINGAPORE 120205
468) 414 COMMONWEALTH AVENUE WES

621) 209 JURONG EAST STREET 21 SINGAPORE 600209
622) 324 JURONG EAST STREET 31 SINGAPORE 600324
623) 331 JURONG EAST AVENUE 1 SINGAPORE 600331
624) 405 PANDAN GARDENS SINGAPORE 600405
625) 403 PANDAN GARDENS SINGAPORE 600403
626) 404 PANDAN GARDENS SINGAPORE 600404
627) 406 PANDAN GARDENS SINGAPORE 600406
628) 337 JURONG EAST AVENUE 1 SINGAPORE 600337
629) 335 JURONG EAST AVENUE 1 SINGAPORE 600335
630) 327 JURONG EAST STREET 31 SAINT LUKE'S ELDERCARE JURONG EAST CENTRE SINGAPORE 600327
631) 323 JURONG EAST STREET 31 SINGAPORE 600323
632) 164 YUNG KUANG ROAD YUNG KUANG COURT SINGAPORE 610164
633) 164 YUNG KUANG ROAD YUNG KUANG COURT SINGAPORE 610164
634) 111 HO CHING ROAD SINGAPORE 610111
635) 114 HO CHING ROAD SINGAPORE 610114
636) 119 HO CHING ROAD SINGAPORE 610119
637) 120 HO CHING ROAD SINGAPORE 610120
638) 3 HU CHING ROAD SINGAPORE 619651
639) 268C BOON LAY DRIVE BOON LAY COURT SINGAPORE 643268
640) 268C BOON LAY DRIVE BOON LAY COURT SINGAPORE 643268
641) 268C BOON LAY DRIVE BOON L

775) 12 MARINE TERRACE MARINE TERRACE BREEZE SINGAPORE 440012
776) 13 MARINE TERRACE MARINE TERRACE BREEZE SINGAPORE 440013
777) 14 MARINE TERRACE MARINE TERRACE BREEZE SINGAPORE 440014
778) 82 COMMONWEALTH CLOSE COMMONWEALTH HEIGHTS SINGAPORE 140082
779) 83 COMMONWEALTH CLOSE COMMONWEALTH HEIGHTS SINGAPORE 140083
780) 63 COMMONWEALTH DRIVE COMMONWEALTH GREEN SINGAPORE 140063
781) 66 COMMONWEALTH DRIVE COMMONWEALTH GREEN SINGAPORE 140066
782) 18 TANGLIN HALT ROAD TANGLIN HALT COURTYARD SINGAPORE 140018
783) 18 TANGLIN HALT ROAD TANGLIN HALT COURTYARD SINGAPORE 140018
784) 111 COMMONWEALTH CRESCENT SINGAPORE 140111
785) 55 COMMONWEALTH DRIVE COMMONWEALTH GREEN SINGAPORE 141055
786) 43 TANGLIN HALT ROAD TANGLIN HALT GREEN SINGAPORE 142043
787) 38 TANGLIN HALT ROAD TANGLIN HALT GREEN SINGAPORE 140038
788) 33 TANGLIN HALT ROAD TANGLIN HALT GREEN SINGAPORE 142033
789) 36 TANGLIN HALT ROAD TANGLIN HALT GREEN SINGAPORE 141036
790) 26 TANGLIN HALT ROAD TANGLIN GROVE SINGAPORE 141026
791) 31 TA

934) 928 TAMPINES STREET 91 TAMPINES PALMSPRING SINGAPORE 520928
935) 832 TAMPINES STREET 82 TAMPINES PALMWALK SINGAPORE 520832
936) 838 TAMPINES STREET 82 TAMPINES PALMWALK SINGAPORE 520838
937) 839 TAMPINES STREET 83 TAMPINES PALMWALK SINGAPORE 520839
938) 248 TAMPINES STREET 21 SINGAPORE 521248
939) 247 TAMPINES STREET 21 SINGAPORE 521247
940) 159 TAMPINES STREET 12 SINGAPORE 521159
941) 812 TAMPINES AVENUE 4 TAMPINES POLYVIEW SINGAPORE 520812
942) 811 TAMPINES AVENUE 4 TAMPINES POLYVIEW SINGAPORE 520811
943) 294 TAMPINES STREET 22 SINGAPORE 520294
944) 432 TAMPINES STREET 41 SINGAPORE 520432
945) 160 TAMPINES STREET 12 SINGAPORE 521160
946) 257 TAMPINES STREET 21 SINGAPORE 520257
947) 816 TAMPINES AVENUE 4 TAMPINES POLYVIEW SINGAPORE 520816
948) 289 TAMPINES STREET 22 SINGAPORE 520289
949) 298 TAMPINES STREET 22 SINGAPORE 520298
950) 232 TAMPINES STREET 21 SINGAPORE 521232
951) 216 TAMPINES STREET 23 SINGAPORE 520216
952) 106 TAMPINES STREET 11 SINGAPORE 521106
953) 912 TAMPINES ST

<b>Total time used to retrieve 1000 addresses: 50 seconds</b>

### 04.3 Using aiohttp

In [21]:
# 'start' here indicates the starting time of the retrieval process
start = time.time()

# Define a coroutine with async def
async def get_response(session, address, count): 
    
    # Using the session created, connect with OneMap API with get() function to get the information from the API
    # await is added as we need to wait the call to complete in the event loop and return a response, else it won't do anything
    response = await session.get(url.format(address))
    
    # Parse the JSON response using .json() method 
    # await is addded since the response is an awaitable object
    data = await response.json()
    
    print('{}) {}'.format(count, data['results'][0]['ADDRESS']))
    
# Since inside the main() function involve coroutines, add 'async' in front of 'def'
# This tells Python that this function can do asynchronous stuff
async def main(): 
    
    count = 1
    tasks = []
    
    # A client session is created to allow the underlying TCP connection to be reused for all requests
    # Using context manager 'with' statement will close the client session automatically when all requests are done
    # Since inside this session involve coroutines, add 'async' in front of 'with'
    async with aiohttp.ClientSession() as session:
        
        for address in addresses[:1000]:
            
            # For each address, create a task of running get_coordinates coroutine using asyncio.create_task()
            # A task will be executed asap whenever there is a stall (await) ongoing
            # When the task is stalled, it will go back to the main caller and run other stuff
            # When the main caller stuff is stalled, it will come back to this task and run the remaining stuff
            # So in our case, while waiting for a response to return, it would start other tasks (making other API calls)
            # Remember to pass the session into get_coordinates coroutine so that a single session is used for all requests
            # Create a list of tasks using .append method
            tasks.append(asyncio.create_task(get_response(session, address, count)))
            count += 1
        
        # After creating a list of tasks, gather all the tasks created using asyncio.gather()
        # Since the tasks is inside a list, use *tasks to unpack the tasks inside the list
        # await is added to execute all the task and wait them to finish
        # Store the returned values in variable 'result'
        await asyncio.gather(*tasks)

# Run the event loop (the loop of an asynchronous operation) by running main()
asyncio.run(main())

# 'end' here indicates the ending time of the retrieval process
end = time.time()

print('Total time used: {} seconds'.format(end - start))

1) 216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPORE 560216
86) 344 ANG MO KIO AVENUE 3 TECK GHEE EVERGREEN SINGAPORE 560344
95) 573 ANG MO KIO AVENUE 3 CHENG SAN COURT SINGAPORE 560573
47) 570 ANG MO KIO AVENUE 3 CHENG SAN COURT SINGAPORE 560570
76) 333 ANG MO KIO AVENUE 1 TECK GHEE VIEW SINGAPORE 560333
77) 156 ANG MO KIO AVENUE 4 YIO CHU KANG GROVE SINGAPORE 560156
14) 230 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW SINGAPORE 560230
8) 220 ANG MO KIO AVENUE 1 ANG MO KIO GROVE SINGAPORE 560220
2) 216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPORE 560216
79) 178 ANG MO KIO AVENUE 4 KEBUN BARU LINK 2 SINGAPORE 560178
20) 345 ANG MO KIO AVENUE 3 TECK GHEE EVERGREEN SINGAPORE 560345
10) 329 ANG MO KIO AVENUE 3 ANG MO KIO 31 SINGAPORE 560329
16) 313 ANG MO KIO AVENUE 3 TECK GHEE EVERGREEN SINGAPORE 560313
82) 231 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW SINGAPORE 560231
19) 343 ANG MO KIO AVENUE 3 MY FIRST SKOOL SINGAPORE 560343
83) 254 ANG MO KIO AVENUE 4 KEBUN BARU VIEW SINGAPORE 560254
18) 110

170) 536 BEDOK NORTH STREET 3 SINGAPORE 460536
174) 702 BEDOK RESERVOIR ROAD RESERVOIR RISE SINGAPORE 470702
171) 537 BEDOK NORTH STREET 3 SINGAPORE 460537
172) 44 CHAI CHEE STREET PING YI GARDENS SINGAPORE 461044
175) 615 BEDOK RESERVOIR ROAD EUNOS GROVE SINGAPORE 470615
181) 15 BEDOK SOUTH ROAD SINGAPORE 460015
182) 34 BEDOK SOUTH AVENUE 2 BEDOK SOUTH PARKVIEW SINGAPORE 460034
176) 712 BEDOK RESERVOIR ROAD BEDOK RESERVOIR GARDEN SINGAPORE 470712
173) 540 BEDOK NORTH STREET 3 BEDOK NORTH GREEN SINGAPORE 460540
177) 50 CHAI CHEE STREET CHAI CHEE GARDENS SINGAPORE 461050
183) 36 BEDOK SOUTH AVENUE 2 BEDOK SOUTH PARKVIEW SINGAPORE 460036
192) 166 BEDOK SOUTH AVENUE 3 SIGLAP EAST SINGAPORE 460166
259) 127 BUKIT BATOK WEST AVENUE 6 SINGAPORE 650127
37) 444 ANG MO KIO AVENUE 10 SINGAPORE 560444
198) 134 BEDOK NORTH STREET 2 SINGAPORE 460134
247) 725 BEDOK RESERVOIR ROAD RESERVOIR VIEW SINGAPORE 470725
197) 80 BEDOK NORTH ROAD SINGAPORE 460080
153) 412 BEDOK NORTH AVENUE 2 SINGAPORE 460412
1

438) 346 CLEMENTI AVENUE 5 SINGAPORE 120346
439) 431 CLEMENTI AVENUE 3 SINGAPORE 120431
459) 411 COMMONWEALTH AVENUE WEST SINGAPORE 120411
435) 305 CLEMENTI AVENUE 4 CLEMENTI MEADOWS SINGAPORE 120305
365) 114 BUKIT PURMEI ROAD BUKIT PURMEI VILLE SINGAPORE 090114
440) 339 CLEMENTI AVENUE 5 SINGAPORE 120339
360) 120 BUKIT MERAH VIEW BUKIT MERAH VIEW SINGAPORE 152120
423) 309A CLEMENTI AVENUE 4 CLEMENTI MEADOWS SINGAPORE 121309
465) 707 CLEMENTI WEST STREET 2 SINGAPORE 120707
451) 713 CLEMENTI WEST STREET 2 SINGAPORE 120713
449) 711 CLEMENTI WEST STREET 2 SINGAPORE 120711
448) 704 WEST COAST ROAD SINGAPORE 120704
434) 307 CLEMENTI AVENUE 4 CLEMENTI MEADOWS SINGAPORE 120307
447) 703 WEST COAST ROAD SINGAPORE 120703
432) 310 CLEMENTI AVENUE 4 CLEMENTI MEADOWS SINGAPORE 120310
461) 311 CLEMENTI AVENUE 4 TRIVELIS SINGAPORE 120311
446) 702 WEST COAST ROAD SINGAPORE 120702
466) 371 CLEMENTI AVENUE 4 SINGAPORE 120371
430) 416A CLEMENTI AVENUE 1 CASA CLEMENTI SINGAPORE 121416
348) 18 TAMAN HO SWE

567) 154 HOUGANG STREET 11 SINGAPORE 530154
568) 341 HOUGANG AVENUE 7 SINGAPORE 530341
559) 248 HOUGANG AVENUE 3 SINGAPORE 530248
558) 252 HOUGANG AVENUE 3 SINGAPORE 530252
538) 2 HOUGANG AVENUE 3 SINGAPORE 530002
537) 13 EUNOS CRESCENT SINGAPORE 400013
555) 308 HOUGANG AVENUE 5 SINGAPORE 530308
501) 128 GEYLANG EAST AVENUE 1 GEYLANG EAST GROVE SINGAPORE 380128
535) 411 EUNOS ROAD 5 SINGAPORE 400411
500) 41 SIMS DRIVE SIMS VISTA SINGAPORE 380041
499) 98 ALJUNIED CRESCENT SINGAPORE 380098
553) 319 HOUGANG AVENUE 5 SINGAPORE 530319
563) 142 LORONG AH SOO SINGAPORE 530142
562) 239 HOUGANG STREET 22 HOUGANG HEIGHTS SINGAPORE 530239
591) 329 JURONG EAST AVENUE 1 SINGAPORE 600329
594) 302 JURONG EAST STREET 32 HONG KAH EAST GARDEN SINGAPORE 600302
566) 309 HOUGANG AVENUE 5 SINGAPORE 530309
565) 137 LORONG AH SOO SINGAPORE 530137
519) 47 CIRCUIT ROAD MACPHERSON GARDEN SINGAPORE 370047
508) 12 PINE CLOSE SINGAPORE 391012
608) 307 JURONG EAST STREET 32 HONG KAH EAST PLACE SINGAPORE 600307
609) 

850) 14 DOVER CLOSE EAST DOVER HEIGHTS SINGAPORE 130014
837) 28 TANGLIN HALT ROAD TANGLIN GROVE SINGAPORE 140028
944) 432 TAMPINES STREET 41 SINGAPORE 520432
910) 157 TAMPINES STREET 12 SINGAPORE 521157
883) 924 TAMPINES STREET 91 TAMPINES PALMSPRING SINGAPORE 520924
927) 161 TAMPINES STREET 12 SINGAPORE 521161
832) 77 COVE DRIVE SINGAPORE 098179
943) 294 TAMPINES STREET 22 SINGAPORE 520294
851) 20 HOLLAND DRIVE SINGAPORE 271020
931) 910 TAMPINES STREET 91 TAMPINES PALMSVILLE SINGAPORE 520910
937) 839 TAMPINES STREET 83 TAMPINES PALMWALK SINGAPORE 520839
929) 261 TAMPINES STREET 21 SINGAPORE 520261
903) 234 TAMPINES STREET 21 SINGAPORE 521234
874) 316 SERANGOON AVENUE 2 SINGAPORE 550316
844) 35 TANGLIN HALT ROAD TANGLIN HALT GREEN SINGAPORE 141035
930) 913 TAMPINES STREET 91 TAMPINES PALMSVILLE SINGAPORE 520913
840) 102 COMMONWEALTH CRESCENT CRESCENT VIEW SINGAPORE 140102
925) 163 TAMPINES STREET 12 SINGAPORE 521163
897) 262 TAMPINES STREET 21 SINGAPORE 520262
889) 111 TAMPINES STREET 

<b>Total time used to retrieve 1000 addresses: 2 seconds</b><br>
Note: The result returned is not in sequence. 

### Conclusion

Using requests.get(): 568 seconds<br>
Using requests session: 50 seconds<br>
Using aiohttp: 2 seconds (Note: The result returned is not in sequence)

# 05.  Get Coordinates of 9804 Flats from OneMap API

### 05.1 Using aiohttp - get error at the end

In [22]:
# 'start' here indicates the starting time of the retrieval process
start = time.time()

# Define a coroutine with async def
async def get_response(session, address, count): 
    
    # Using the session created, connect with OneMap API with get() function to get the information from the API
    # await is added as we need to wait the call to complete in the event loop and return a response, else it won't do anything
    response = await session.get(url.format(address))
    
    # Parse the JSON response using .json() method 
    # await is addded since the response is an awaitable object
    data = await response.json()
    
    print('{}) {}'.format(count, data['results'][0]['ADDRESS']))
    
# Since inside the main() function involve coroutines, add 'async' in front of 'def'
# This tells Python that this function can do asynchronous stuff
async def main(): 
    
    count = 1
    tasks = []
    
    # A client session is created to allow the underlying TCP connection to be reused for all requests
    # Using context manager 'with' statement will close the client session automatically when all requests are done
    # Since inside this session involve coroutines, add 'async' in front of 'with'
    async with aiohttp.ClientSession() as session:
        
        for address in addresses:
            
            # For each address, create a task of running get_coordinates coroutine using asyncio.create_task()
            # A task will be executed asap whenever there is a stall (await) ongoing
            # When the task is stalled, it will go back to the main caller and run other stuff
            # When the main caller stuff is stalled, it will come back to this task and run the remaining stuff
            # So in our case, while waiting for a response to return, it would start other tasks (making other API calls)
            # Remember to pass the session into get_coordinates coroutine so that a single session is used for all requests
            # Create a list of tasks using .append method
            tasks.append(asyncio.create_task(get_response(session, address, count)))
            count += 1
        
        # After creating a list of tasks, gather all the tasks created using asyncio.gather()
        # Since the tasks is inside a list, use *tasks to unpack the tasks inside the list
        # await is added to execute all the task and wait them to finish
        # Store the returned values in variable 'result'
        await asyncio.gather(*tasks)

# Run the event loop (the loop of an asynchronous operation) by running main()
asyncio.run(main())

# 'end' here indicates the ending time of the retrieval process
end = time.time()

print('Total time used: {} seconds'.format(end - start))

52) 564 ANG MO KIO AVENUE 3 SINGAPORE 560564
53) 560 ANG MO KIO AVENUE 10 SINGAPORE 560560
48) 586 ANG MO KIO AVENUE 3 CHENG SAN COURT SINGAPORE 560586
47) 570 ANG MO KIO AVENUE 3 CHENG SAN COURT SINGAPORE 560570
46) 474 ANG MO KIO AVENUE 10 TECK GHEE HORIZON SINGAPORE 560474
54) 558 ANG MO KIO AVENUE 10 SINGAPORE 560558
45) 471 ANG MO KIO AVENUE 10 TECK GHEE HORIZON SINGAPORE 560471
44) 466 ANG MO KIO AVENUE 10 PCF SPARKLETOTS PRESCHOOL @ TECK GHEE BLK 466 (KN) SINGAPORE 560466
55) 559 ANG MO KIO AVENUE 10 CHENG SAN PLACE SINGAPORE 560559
43) 425 ANG MO KIO AVENUE 3 CHONG BOON VIEW SINGAPORE 560425
42) 424 ANG MO KIO AVENUE 3 CHONG BOON VIEW SINGAPORE 560424
41) 434 ANG MO KIO AVENUE 10 CHONG BOON RISE SINGAPORE 560434
2) 216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPORE 560216
40) 433 ANG MO KIO AVENUE 10 CHONG BOON RISE SINGAPORE 560433
36) 443 ANG MO KIO AVENUE 10 SINGAPORE 560443
32) 418 ANG MO KIO AVENUE 10 TECK GHEE HEARTLANDS SINGAPORE 560418
8) 220 ANG MO KIO AVENUE 1 ANG MO KIO

247) 725 BEDOK RESERVOIR ROAD RESERVOIR VIEW SINGAPORE 470725
245) 139 BEDOK NORTH AVENUE 3 SINGAPORE 460139
280) 169 BUKIT BATOK WEST AVENUE 8 SINGAPORE 650169
234) 112 BEDOK RESERVOIR ROAD EUNOS VISTA SINGAPORE 470112
184) 35 BEDOK SOUTH AVENUE 2 BEDOK SOUTH PARKVIEW SINGAPORE 460035
201) 132 BEDOK NORTH STREET 2 SINGAPORE 460132
178) 54 CHAI CHEE STREET CHAI CHEE GARDENS SINGAPORE 460054
266) 146 BUKIT BATOK WEST AVENUE 6 SINGAPORE 650146
183) 36 BEDOK SOUTH AVENUE 2 BEDOK SOUTH PARKVIEW SINGAPORE 460036
219) 708 BEDOK NORTH ROAD RESERVOIR RISE SINGAPORE 470708
212) 95 BEDOK NORTH AVENUE 4 SINGAPORE 460095
249) 24 SIN MING ROAD SIN MING VILLE SINGAPORE 570024
189) 8 BEDOK SOUTH AVENUE 2 SINGAPORE 460008
243) 96 BEDOK NORTH AVENUE 4 SINGAPORE 460096
197) 80 BEDOK NORTH ROAD SINGAPORE 460080
269) 143 BUKIT BATOK STREET 11 SINGAPORE 650143
193) 71 BEDOK SOUTH ROAD SIGLAP LUCKY VALLEY SINGAPORE 460071
199) 126 BEDOK NORTH STREET 2 SINGAPORE 460126
230) 613 BEDOK RESERVOIR ROAD EUNOS GRO

416) 427 CLEMENTI AVENUE 3 SINGAPORE 120427
417) 334 CLEMENTI AVENUE 2 SINGAPORE 120334
419) 323 HOUGANG AVENUE 5 SINGAPORE 530323
414) 363 CLEMENTI AVENUE 2 SINGAPORE 120363
428) 506 WEST COAST DRIVE WEST COAST VISTA SINGAPORE 120506
423) 309A CLEMENTI AVENUE 4 CLEMENTI MEADOWS SINGAPORE 121309
434) 307 CLEMENTI AVENUE 4 CLEMENTI MEADOWS SINGAPORE 120307
427) 433 CLEMENTI AVENUE 3 SINGAPORE 120433
413) 509 WEST COAST DRIVE STAR KIDSCAMPUS SINGAPORE 120509
425) 354 CLEMENTI AVENUE 2 CLEMENTI AVENUE 2 SHOPPING CENTRE SINGAPORE 120354
430) 416A CLEMENTI AVENUE 1 CASA CLEMENTI SINGAPORE 121416
505) 106 ALJUNIED CRESCENT SINGAPORE 380106
510) 65 CIRCUIT ROAD SINGAPORE 370065
433) 309 CLEMENTI AVENUE 4 CLEMENTI MEADOWS SINGAPORE 120309
429) 508 WEST COAST DRIVE WEST COAST VISTA SINGAPORE 120508
517) 57 CIRCUIT ROAD SINGAPORE 370057
512) 23 BALAM ROAD BALAM GARDENS SINGAPORE 370023
459) 411 COMMONWEALTH AVENUE WEST SINGAPORE 120411
518) 58 CIRCUIT ROAD MACPHERSON GARDEN SINGAPORE 370058
442)

673) 481 JURONG WEST STREET 41 JURONGVILLE @ STREET 41 SINGAPORE 640481
688) 543 JURONG WEST STREET 42 SINGAPORE 640543
666) 527 JURONG WEST STREET 52 SINGAPORE 640527
628) 337 JURONG EAST AVENUE 1 SINGAPORE 600337
632) 164 YUNG KUANG ROAD YUNG KUANG COURT SINGAPORE 610164
592) 308 JURONG EAST STREET 32 HONG KAH EAST PLACE SINGAPORE 600308
608) 307 JURONG EAST STREET 32 HONG KAH EAST PLACE SINGAPORE 600307
684) 450 JURONG WEST STREET 42 SINGAPORE 640450
563) 142 LORONG AH SOO SINGAPORE 530142
574) 302 HOUGANG AVENUE 5 SINGAPORE 530302
680) 447 JURONG WEST STREET 42 SINGAPORE 640447
598) 244 JURONG EAST STREET 24 SINGAPORE 600244
681) 554 JURONG WEST STREET 42 SINGAPORE 640554
604) 210 JURONG EAST STREET 21 SINGAPORE 600210
647) 208 BOON LAY PLACE BOON LAY PLACE SINGAPORE 640208
682) 433 JURONG WEST STREET 42 SINGAPORE 640433
649) 210 BOON LAY PLACE BOON LAY PLACE SINGAPORE 640210
678) 448 JURONG WEST STREET 42 SINGAPORE 640448
573) 339 HOUGANG AVENUE 7 SINGAPORE 530339
576) 140 LORONG 

829) 59 COMMONWEALTH DRIVE COMMONWEALTH GREEN SINGAPORE 140059
838) 98 COMMONWEALTH CRESCENT CRESCENT VIEW SINGAPORE 140098
832) 77 COVE DRIVE SINGAPORE 098179
858) 115 SERANGOON NORTH AVENUE 1 HWI YOH VILLE SINGAPORE 550115
871) 212 SERANGOON AVENUE 4 SINGAPORE 550212
836) 25 TANGLIN HALT ROAD TANGLIN GROVE SINGAPORE 140025
820) 16 MOONBEAM DRIVE HOLLAND GARDENS SINGAPORE 277354
840) 102 COMMONWEALTH CRESCENT CRESCENT VIEW SINGAPORE 140102
875) 102 TAMPINES STREET 11 SINGAPORE 521102
878) 209 TAMPINES STREET 21 SINGAPORE 520209
881) 202 TAMPINES STREET 21 SINGAPORE 520202
874) 316 SERANGOON AVENUE 2 SINGAPORE 550316
882) 138 TAMPINES STREET 11 SINGAPORE 521138
885) 817 TAMPINES STREET 81 PICKET FENCE @ TAMPINES PTE. LTD. SINGAPORE 520817
883) 924 TAMPINES STREET 91 TAMPINES PALMSPRING SINGAPORE 520924
897) 262 TAMPINES STREET 21 SINGAPORE 520262
899) 150 TAMPINES STREET 12 SINGAPORE 521150
876) 113 TAMPINES STREET 11 SINGAPORE 521113
835) 113 COMMONWEALTH CRESCENT SINGAPORE 140113
854

...

4533) 486B TAMPINES AVENUE 9 SINGAPORE 521486
4531) 491B TAMPINES AVENUE 9 SINGAPORE 521491
4509) 212 CHOA CHU KANG CENTRAL SINGAPORE 680212
4529) 488A TAMPINES AVENUE 9 SINGAPORE 520488
4545) 259 BISHAN STREET 22 SINGAPORE 570259
4560) 338 HOUGANG AVENUE 7 SINGAPORE 530338
4495) 115 LENGKONG TIGA SINGAPORE 410115
4521) 953 JURONG WEST STREET 91 SINGAPORE 640953
4544) 152 BISHAN STREET 11 BISHAN GREEN SINGAPORE 570152
4553) 274 CHOA CHU KANG AVENUE 2 KEAT HONG TEMBUSU SINGAPORE 680274
4543) 129 BISHAN STREET 12 SINGAPORE 570129
4558) 361 HOUGANG AVENUE 5 SINGAPORE 530361
4561) 646 HOUGANG AVENUE 8 SINGAPORE 530646
4546) 420 FAJAR ROAD SINGAPORE 670420
4559) 540 HOUGANG AVENUE 8 SINGAPORE 530540
4520) 431 HOUGANG AVENUE 6 OASIS @ HOUGANG SINGAPORE 530431
4542) 105 LENGKONG TIGA KEMBANGAN GARDENS SINGAPORE 410105
4550) 135 TECK WHYE LANE SINGAPORE 680135
4549) 229 CHOA CHU KANG CENTRAL SINGAPORE 680229
4541) 805 CHAI CHEE ROAD PING-AN GARDENS SINGAPORE 460805
4551) 113 TECK WHYE LANE SIN

4780) 140 PASIR RIS STREET 11 SINGAPORE 510140
4737) 293 CHOA CHU KANG AVENUE 3 KEAT HONG TEMBUSU SINGAPORE 680293
4765) 950 JURONG WEST STREET 91 SINGAPORE 640950
4772) 816 WOODLANDS STREET 82 SINGAPORE 730816
4748) 407 PASIR RIS DRIVE 6 SINGAPORE 510407
4770) 413 PASIR RIS DRIVE 6 SINGAPORE 510413
4777) 427 HOUGANG AVENUE 6 OASIS @ HOUGANG SINGAPORE 530427
4746) 114 WHAMPOA ROAD HAVEN INFANT & TODDLER CENTRE SINGAPORE 320114
4747) 406 PASIR RIS DRIVE 6 SINGAPORE 510406
4787) 220 CHOA CHU KANG CENTRAL SINGAPORE 680220
4795) 362 HOUGANG AVENUE 5 SINGAPORE 530362
4752) 454 PASIR RIS DRIVE 6 SINGAPORE 510454
4790) 406 CHOA CHU KANG AVENUE 3 KEAT HONG LAUREL SINGAPORE 680406
4797) 830 JURONG WEST STREET 81 SINGAPORE 640830
4792) 422 HOUGANG AVENUE 6 HDB PUBLIC SHELTERS SINGAPORE 530422
4781) 759 PASIR RIS STREET 71 SINGAPORE 510759
4782) 218 PASIR RIS STREET 21 SINGAPORE 510218
4794) 451 HOUGANG AVENUE 10 SINGAPORE 530451
4727) 601 ELIAS ROAD ELIAS VIEW SINGAPORE 510601
4791) 410 EUNOS RO

4992) 557 PASIR RIS STREET 51 SINGAPORE 510557
4994) 724 PASIR RIS STREET 72 SINGAPORE 510724
4993) 147 PASIR RIS STREET 13 SINGAPORE 510147
1126) 476 ANG MO KIO AVENUE 10 TECK GHEE HORIZON SINGAPORE 560476
4970) 533 SERANGOON NORTH AVENUE 4 SINGAPORE 550533
4998) 605 ELIAS ROAD ELIAS VIEW SINGAPORE 510605
1129) 612 ANG MO KIO AVENUE 4 YIO CHU KANG GREEN SINGAPORE 560612
5000) 536 SERANGOON NORTH AVENUE 4 SINGAPORE 550536
5002) 532 SERANGOON NORTH AVENUE 4 SINGAPORE 550532
4977) 661 YISHUN AVENUE 4 NEE SOON CENTRAL VIEW SINGAPORE 760661
4989) 273 PASIR RIS STREET 21 SINGAPORE 510273
4971) 541 SERANGOON NORTH AVENUE 4 SINGAPORE 550541
4996) 457 PASIR RIS DRIVE 4 SINGAPORE 510457
4986) 221 HOUGANG STREET 21 HDB PUBLIC SHELTERS SINGAPORE 530221
4981) 251 CHOA CHU KANG AVENUE 2 SINGAPORE 680251
5010) 423 ANG MO KIO AVENUE 3 CHENG SAN FIRE POST SINGAPORE 560423
4988) 478 PASIR RIS DRIVE 4 SINGAPORE 510478
5012) 663 JALAN DAMAI EUNOS DAMAI VILLE SINGAPORE 410663
5016) 253 BISHAN STREET 22 SI

5174) 262 BISHAN STREET 22 SINGAPORE 570262
5176) 202 BISHAN STREET 23 SINGAPORE 570202
5078) 552 CHOA CHU KANG STREET 52 SINGAPORE 680552
5139) 497J TAMPINES STREET 45 SINGAPORE 527497
5159) 733 JURONG WEST STREET 73 SINGAPORE 640733
5188) 737 TAMPINES STREET 72 TAMPINES GREEN VALE SINGAPORE 520737
1439) 833 TAMPINES STREET 83 TAMPINES PALMWALK SINGAPORE 520833
5130) 131 PASIR RIS STREET 11 SINGAPORE 510131
5201) 832 HOUGANG CENTRAL SINGAPORE 530832
5183) 804 HOUGANG CENTRAL SINGAPORE 530804
5171) 419 WOODLANDS STREET 41 SINGAPORE 730419
5137) 472 TAMPINES STREET 44 SINGAPORE 520472
5205) 539 SERANGOON NORTH AVENUE 4 SINGAPORE 550539
5210) 102 ANG MO KIO AVENUE 3 CHILD AT STREET 11 LTD SINGAPORE 560102
5209) 927 YISHUN CENTRAL 1 SINGAPORE 760927
5185) 222 PASIR RIS STREET 21 SINGAPORE 510222
1467) 175 LORONG 2 TOA PAYOH TOA PAYOH CENTRAL SINGAPORE 310175
5206) 525 SERANGOON NORTH AVENUE 4 SINGAPORE 550525
1441) 114 TAMPINES STREET 11 SINGAPORE 521114
5161) 250 PASIR RIS STREET 21 SING

ContentTypeError: 0, message='Attempt to decode JSON with unexpected mimetype: text/html; charset=utf-8', url=URL('https://www.onemap.gov.sg/api/common/elastic/search?searchVal=558+PASIR+RIS+ST+51&returnGeom=Y&getAddrDetails=Y&pageNum=1')

<b>When the amount of API calls are large, error occurs. This could be due to the rate limit of OneMap API. </b>

### 05.2 Solution - Using aiohttp with Semaphore (rate limiter)

In [23]:
# 'start' here indicates the starting time of the retrieval process
start = time.time()

# Define a coroutine with async def
async def get_response(session, semaphore, address, count): 
    
    # Use async with statement to utilize the Semaphore
    async with semaphore: 
    
        # Using the session created, connect with OneMap API with get() function to get the information from the API
        # await is added as we need to wait the call to complete in the event loop and return a response, else it won't do anything
        response = await session.get(url.format(address))

        # Parse the JSON response using .json() method 
        # await is addded since the response is an awaitable object
        data = await response.json()

        print('{}) {}'.format(count, data['results'][0]['ADDRESS']))
    
# Since inside the main() function involve coroutines, add 'async' in front of 'def'
# This tells Python that this function can do asynchronous stuff
async def main(): 
    
    count = 1
    tasks = []
    
    # Initialize a Semaphore 
    # If Semaphore value is 20, max 20 API calls would be made at the same time. Only 20 calls would be in queue. 
    semaphore = asyncio.Semaphore(20)
    
    # A client session is created to allow the underlying TCP connection to be reused for all requests
    # Using context manager 'with' statement will close the client session automatically when all requests are done
    # Since inside this session involve coroutines, add 'async' in front of 'with'
    async with aiohttp.ClientSession() as session:
        
        for address in addresses:
            
            # For each address, create a task of running get_coordinates coroutine using asyncio.create_task()
            # A task will be executed asap whenever there is a stall (await) ongoing
            # When the task is stalled, it will go back to the main caller and run other stuff
            # When the main caller stuff is stalled, it will come back to this task and run the remaining stuff
            # So in our case, while waiting for a response to return, it would start other tasks (making other API calls)
            # Remember to pass the session into get_coordinates coroutine so that a single session is used for all requests
            # Create a list of tasks using .append method
            tasks.append(asyncio.create_task(get_response(session, semaphore, address, count)))
            count += 1
        
        # After creating a list of tasks, gather all the tasks created using asyncio.gather()
        # Since the tasks is inside a list, use *tasks to unpack the tasks inside the list
        # await is added to execute all the task and wait them to finish
        # Store the returned values in variable 'result'
        await asyncio.gather(*tasks)

# Run the event loop (the loop of an asynchronous operation) by running main()
asyncio.run(main())

# 'end' here indicates the ending time of the retrieval process
end = time.time()

print('Total time used: {} seconds'.format(end - start))

2) 216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPORE 560216
9) 219 ANG MO KIO AVENUE 1 ANG MO KIO GROVE SINGAPORE 560219
19) 343 ANG MO KIO AVENUE 3 MY FIRST SKOOL SINGAPORE 560343
4) 202 ANG MO KIO AVENUE 3 ANG MO KIO VIEW SINGAPORE 560202
6) 232 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW SINGAPORE 560232
1) 216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPORE 560216
14) 230 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW SINGAPORE 560230
10) 329 ANG MO KIO AVENUE 3 ANG MO KIO 31 SINGAPORE 560329
7) 308C ANG MO KIO AVENUE 1 TECK GHEE VISTA SINGAPORE 563308
5) 235 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW SINGAPORE 560235
17) 117 ANG MO KIO AVENUE 4 KEBUN BARU HEIGHTS SINGAPORE 560117
18) 110 ANG MO KIO AVENUE 4 KEBUN BARU HEIGHTS SINGAPORE 560110
15) 329 ANG MO KIO AVENUE 3 ANG MO KIO 31 SINGAPORE 560329
13) 223 ANG MO KIO AVENUE 1 SINGAPORE 560223
11) 320 ANG MO KIO AVENUE 1 STAR LEARNERS @ ANG MO KIO PTE. LTD. SINGAPORE 560320
16) 313 ANG MO KIO AVENUE 3 TECK GHEE EVERGREEN SINGAPORE 560313
21) 346 

160) 419 BEDOK NORTH STREET 1 GARDEN HILL SINGAPORE 460419
168) 532 BEDOK NORTH STREET 3 SINGAPORE 460532
174) 702 BEDOK RESERVOIR ROAD RESERVOIR RISE SINGAPORE 470702
175) 615 BEDOK RESERVOIR ROAD EUNOS GROVE SINGAPORE 470615
170) 536 BEDOK NORTH STREET 3 SINGAPORE 460536
177) 50 CHAI CHEE STREET CHAI CHEE GARDENS SINGAPORE 461050
173) 540 BEDOK NORTH STREET 3 BEDOK NORTH GREEN SINGAPORE 460540
171) 537 BEDOK NORTH STREET 3 SINGAPORE 460537
172) 44 CHAI CHEE STREET PING YI GARDENS SINGAPORE 461044
179) 606 BEDOK RESERVOIR ROAD EUNOS RAINBOW SINGAPORE 470606
176) 712 BEDOK RESERVOIR ROAD BEDOK RESERVOIR GARDEN SINGAPORE 470712
178) 54 CHAI CHEE STREET CHAI CHEE GARDENS SINGAPORE 460054
182) 34 BEDOK SOUTH AVENUE 2 BEDOK SOUTH PARKVIEW SINGAPORE 460034
181) 15 BEDOK SOUTH ROAD SINGAPORE 460015
186) 42 BEDOK SOUTH ROAD BEDOK SOUTH PARKVIEW SINGAPORE 460042
180) 616 BEDOK RESERVOIR ROAD EUNOS GROVE SINGAPORE 470616
183) 36 BEDOK SOUTH AVENUE 2 BEDOK SOUTH PARKVIEW SINGAPORE 460036
184) 35

332) 62 TELOK BLANGAH HEIGHTS BLANGAH GARDEN SINGAPORE 100062
337) 141 JALAN BUKIT MERAH SINGAPORE 160141
335) 55 TELOK BLANGAH DRIVE BLANGAH VIEW SINGAPORE 100055
334) 60 TELOK BLANGAH HEIGHTS BLANGAH GARDEN SINGAPORE 100060
340) 146 JALAN BUKIT MERAH SINGAPORE 160146
333) 63 TELOK BLANGAH HEIGHTS BLANGAH GARDEN SINGAPORE 100063
336) 75 TELOK BLANGAH DRIVE SINGAPORE 100075
339) 3 EVERTON PARK EVERTON PARK SINGAPORE 080003
341) 541 KALLANG BAHRU GEYLANG BAHRU MRT STATION (DT24) SINGAPORE 339355
338) 7 EVERTON PARK SINGAPORE 080007
342) 86 REDHILL CLOSE SINGAPORE 150086
346) 28 HOY FATT ROAD BRICKWORKS ESTATE SINGAPORE 151028
345) 28 JALAN BUKIT MERAH BRICKWORKS ESTATE SINGAPORE 152028
343) 86 REDHILL CLOSE SINGAPORE 150086
347) 110 ZION ROAD HAVELOCK MRT STATION (TE16) SINGAPORE 168604
348) 18 TAMAN HO SWEE BUKIT HO SWEE VIEW SINGAPORE 162018
349) 30 JALAN KLINIK SINGAPORE 160030
351) 28 JALAN KLINIK SINGAPORE 160028
344) 86 REDHILL CLOSE SINGAPORE 150086
353) 37 JALAN RUMAH TINGGI RUM

499) 98 ALJUNIED CRESCENT SINGAPORE 380098
352) 22 HAVELOCK ROAD SINGAPORE 160022
503) 102 ALJUNIED CRESCENT SINGAPORE 380102
506) 112 ALJUNIED CRESCENT SINGAPORE 380112
504) 99 ALJUNIED CRESCENT SINGAPORE 380099
505) 106 ALJUNIED CRESCENT SINGAPORE 380106
508) 12 PINE CLOSE SINGAPORE 391012
507) 108 ALJUNIED CRESCENT SINGAPORE 380108
510) 65 CIRCUIT ROAD SINGAPORE 370065
511) 34 CIRCUIT ROAD SINGAPORE 370034
509) 22 HAIG ROAD HAIG VISTA SINGAPORE 430022
521) 49 CIRCUIT ROAD MACPHERSON GARDEN SINGAPORE 370049
514) 93 JALAN DUA SINGAPORE 390093
516) 67 CIRCUIT ROAD SINGAPORE 370067
512) 23 BALAM ROAD BALAM GARDENS SINGAPORE 370023
515) 61 CIRCUIT ROAD SINGAPORE 370061
517) 57 CIRCUIT ROAD SINGAPORE 370057
519) 47 CIRCUIT ROAD MACPHERSON GARDEN SINGAPORE 370047
518) 58 CIRCUIT ROAD MACPHERSON GARDEN SINGAPORE 370058
520) 48 CIRCUIT ROAD MACPHERSON GARDEN SINGAPORE 370048
523) 44 CIRCUIT ROAD SINGAPORE 370044
527) 127 GEYLANG EAST AVENUE 1 GEYLANG EAST GROVE SINGAPORE 380127
526) 36 CIRCU

751) 104 JALAN RAJAH SINGAPORE 321104
750) 807 KING GEORGE'S AVENUE SINGAPORE 200807
747) 52 SOUTH BUONA VISTA ROAD KENT RIDGE HILL RESIDENCES SINGAPORE 118168
749) 103 AH HOOD ROAD AH HOOD GARDENS SINGAPORE 320103
752) 39 UPPER BOON KENG ROAD UPPER BOON KENG RIVERVIEW SINGAPORE 380039
753) 461 CRAWFORD LANE CRAWFORD COURT SINGAPORE 190461
754) 56 GEYLANG BAHRU GEYLANG BAHRU RIVERPOINT SINGAPORE 330056
759) 66 MARINE DRIVE MARINE DRIVE GARDENS SINGAPORE 440066
758) 35 MARINE CRESCENT MARINE CRESCENT VILLE SINGAPORE 440035
756) 53 GEYLANG BAHRU GEYLANG BAHRU RIVERPOINT SINGAPORE 330053
757) 12 NORTH BRIDGE ROAD GOLDEN BEACH VISTA SINGAPORE 190012
755) 54 GEYLANG BAHRU GEYLANG BAHRU RIVERPOINT SINGAPORE 330054
760) 67 MARINE DRIVE MARINE DRIVE GARDENS SINGAPORE 440067
761) 4 MARINE TERRACE MARINE PARADE-FOO HAI ELDERLY LODGE SINGAPORE 440004
763) 32 MARINE CRESCENT MARINE CRESCENT VILLE SINGAPORE 440032
764) 31 MARINE CRESCENT MARINE CRESCENT VILLE SINGAPORE 440031
765) 30 MARINE CRESCEN

...

9180) 494D TAMPINES STREET 43 TAMPINES ALCOVES SINGAPORE 524494
9190) 671A EDGEFIELD PLAINS WATERWAY BANKS SINGAPORE 821671
9187) 665C PUNGGOL DRIVE WATERWAY WOODCRESS SINGAPORE 823665
9182) 504B YISHUN STREET 51 ACACIA BREEZE @ YISHUN SINGAPORE 762504
9193) 490B CHOA CHU KANG AVENUE 5 SUNSHINE GARDENS SINGAPORE 682490
9195) 490D CHOA CHU KANG AVENUE 5 SUNSHINE GARDENS SINGAPORE 684490
9192) 468B FERNVALE LINK FERNVALE LEA SINGAPORE 792468
9194) 489D CHOA CHU KANG AVENUE 5 SUNSHINE GARDENS SINGAPORE 684489
9199) 471C FERNVALE STREET FERNVALE RIVERGROVE SINGAPORE 793471
9202) 488B CHOA CHU KANG AVENUE 5 SUNSHINE GARDENS SINGAPORE 682488
9205) 469C SENGKANG WEST WAY FERNVALE LEA SINGAPORE 793469
9196) 308A PUNGGOL WALK GREENLAND CHILDCARE @ PUNGGOL PTE LTD SINGAPORE 821308
9197) 86 DAWSON ROAD SKYVILLE @ DAWSON SINGAPORE 141086
9201) 807A CHAI CHEE ROAD PING YI GREENS SINGAPORE 461807
9198) 90 DAWSON ROAD SKYTERRACE @ DAWSON SINGAPORE 142090
9203) 488C CHOA CHU KANG AVENUE 5 SUNSHINE GAR

9324) 163C RIVERVALE CRESCENT RIVERVALE DELTA SINGAPORE 543163
9326) 215B COMPASSVALE DRIVE COMPASSVALE MAST SINGAPORE 542215
9328) 163A RIVERVALE CRESCENT RIVERVALE DELTA SINGAPORE 541163
9327) 163B RIVERVALE CRESCENT RIVERVALE DELTA SINGAPORE 542163
9336) 808A CHOA CHU KANG AVENUE 1 KEAT HONG AXIS SINGAPORE 681808
9329) 7 UPPER ALJUNIED LANE JOO SENG GREEN SINGAPORE 360007
9337) 815C CHOA CHU KANG AVENUE 7 KEAT HONG QUAD SINGAPORE 683815
9331) 889B WOODLANDS DRIVE 50 TREETRAIL@WOODLANDS SINGAPORE 732889
9333) 70A TELOK BLANGAH HEIGHTS TELOK BLANGAH RIDGEVIEW SINGAPORE 101070
9335) 804A KEAT HONG CLOSE KEAT HONG PRIDE SINGAPORE 681804
9338) 63 SIMS PLACE GREENTOPS @ SIMS PLACE SINGAPORE 380063
9334) 803B KEAT HONG CLOSE KEAT HONG PRIDE SINGAPORE 682803
9339) 697B JURONG WEST CENTRAL 3 JURONG WEST BLOSSOM SINGAPORE 642697
9340) 315A PUNGGOL WAY WATERWAY CASCADIA SINGAPORE 821315
9330) 302A WOODLANDS STREET 31 WOODLANDS PEAK SINGAPORE 731302
9350) 808C CHOA CHU KANG AVENUE 1 KEAT HONG A

9508) 260B ANG MO KIO STREET 21 KEBUN BARU COURT SINGAPORE 562260
9511) 590A ANG MO KIO STREET 51 CHENG SAN COURT SINGAPORE 561590
9509) 226F ANG MO KIO AVENUE 1 KEBUN BARU MALL SINGAPORE 566226
9514) 296C BUKIT BATOK STREET 22 SKYLINE II @ BUKIT BATOK SINGAPORE 653296
9503) 509C YISHUN AVENUE 4 ANGSANA BREEZE @ YISHUN SINGAPORE 763509
9504) 502B YISHUN STREET 51 PALM BREEZE @ YISHUN SINGAPORE 762502
9506) 455A ANG MO KIO STREET 44 TECK GHEE PARKVIEW SINGAPORE 561455
9515) 296A BUKIT BATOK STREET 22 SKYLINE II @ BUKIT BATOK SINGAPORE 651296
9498) 216A BOON LAY AVENUE BOON LAY VIEW SINGAPORE 641216
9507) 455B ANG MO KIO STREET 44 TECK GHEE PARKVIEW SINGAPORE 562455
9510) 591A ANG MO KIO STREET 51 CHENG SAN COURT SINGAPORE 561591
9512) 590B ANG MO KIO STREET 51 CHENG SAN COURT SINGAPORE 562590
9519) 450A BUKIT BATOK WEST AVENUE 6 WEST TERRA @ BUKIT BATOK SINGAPORE 651450
9516) 443D BUKIT BATOK WEST AVENUE 8 WEST RIDGES @ BUKIT BATOK SINGAPORE 654443
9513) 296B BUKIT BATOK STREET 22 SKYLI

9690) 377 BUKIT BATOK STREET 31 BUKIT GOMBAK VISTA SINGAPORE 650377
9682) 186A BEDOK NORTH STREET 4 FENGSHAN GREENVILLE SINGAPORE 461186
9685) 450D BUKIT BATOK WEST AVENUE 6 WEST TERRA @ BUKIT BATOK SINGAPORE 654450
9689) 448B BUKIT BATOK WEST AVENUE 9 WEST VALLEY @ BUKIT BATOK SINGAPORE 652448
9684) 451A BUKIT BATOK WEST AVENUE 6 WEST TERRA @ BUKIT BATOK SINGAPORE 651451
9681) 187B BEDOK NORTH STREET 4 FENGSHAN GREENVILLE SINGAPORE 462187
9693) 292B BUKIT BATOK EAST AVENUE 6 SKYPEAK @ BUKIT BATOK SINGAPORE 652292
9687) 441A BUKIT BATOK WEST AVENUE 8 WEST RIDGES @ BUKIT BATOK SINGAPORE 651441
9686) 440A BUKIT BATOK WEST AVENUE 8 WEST RIDGES @ BUKIT BATOK SINGAPORE 651440
9691) 436A BUKIT BATOK WEST AVENUE 5 WEST EDGE @ BUKIT BATOK SINGAPORE 651436
9695) 435C BUKIT BATOK WEST AVENUE 5 WEST EDGE @ BUKIT BATOK SINGAPORE 653435
9688) 448A BUKIT BATOK WEST AVENUE 9 WEST VALLEY @ BUKIT BATOK SINGAPORE 651448
9696) 451B BUKIT BATOK WEST AVENUE 6 WEST TERRA @ BUKIT BATOK SINGAPORE 652451
9694)

8607) 307B ANG MO KIO AVENUE 1 TECK GHEE VISTA SINGAPORE 562307
8761) 138A LORONG 1A TOA PAYOH THE PEAK @ TOA PAYOH SINGAPORE 311138
Total time used: 38.03344702720642 seconds


<b>Total time used to retrieve 9804 addresses: 38 seconds</b><br>
No error.<br>
Note: The result returned is not in sequence.